In [1]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results;

In [ ]:
# def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
#     mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
#     mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                              mp_drawing.DrawingSpec(color=(224, 227, 73), thickness=1, circle_radius=1), 
                              mp_drawing.DrawingSpec(color=(234, 11, 10), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(234, 11, 10), thickness=2, circle_radius=1), 
                              mp_drawing.DrawingSpec(color=(234, 56, 21), thickness=2, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(234, 11, 10), thickness=1, circle_radius=4), 
                              mp_drawing.DrawingSpec(color=(234, 56, 21), thickness=1, circle_radius=4))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(234, 11, 10), thickness=1, circle_radius=4), 
                              mp_drawing.DrawingSpec(color=(234, 56, 21), thickness=1, circle_radius=4))

In [ ]:
# cap = cv2.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence = 0.6, min_tracking_confidence = 0.5) as holistic:
#     while cap.isOpened():
#         ret, frame = cap.read()
        
#         image, results = mediapipe_detection(frame, holistic) # Detecting
# #         print(results)

#         draw_landmarks(image, results)
#         cv2.imshow('OpenCV Feed', image)
    
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

In [ ]:
# draw_landmarks(frame, results)
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

#### Extract keypoints

In [ ]:
# pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
# face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)

In [ ]:
# lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
# rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

#### Collection of data

In [6]:
DATA_PATH = os.path.join('temp_dataset')

actions = np.array(['hello', 'thanks', 'iloveyou'])
seq = 30 # 30 videos
frmLen = 30 # frames per video

In [8]:
for action in actions:
    for sequence in range(seq):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [9]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.6, min_tracking_confidence = 0.5) as holistic:
    for action in actions:
        for sequence in range(seq):
            for frmNo in range(frmLen):
                ret, frame = cap.read()
        
                image, results = mediapipe_detection(frame, holistic) # Detecting
        #         print(results)

                draw_landmarks(image, results)
                
                if frmNo == 0:
                    cv2.putText(image, 'Starting collection...', (120, 200),
                               cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} video number {}'.format(action, sequence), (120, 200),
                               cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} video number {}'.format(action, sequence), (120, 200),
                               cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frmNo))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

#### Preprocessing

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [8]:
label_map = {label : num for num, label in enumerate(actions)}

In [12]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [21]:
sequences, labels = [], []
for action in actions:
    for sequence in range(seq):
        window = []
        for frmNo in range(frmLen):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frmNo)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
# np.array(sequences).shape

In [22]:
x = np.array(sequences)

In [23]:
y = to_categorical(labels).astype(int)

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.05)

In [ ]:
# len(y_test)

#### Training...

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [10]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [13]:
from tensorflow.compat.v1.train import Optimizer
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [21]:
model.fit(x_train, y_train, epochs=1600, callbacks=[tb_callback])

Epoch 1/2000


3/3 [==============================] - 3s 39ms/step - loss: 1.0893 - categorical_accuracy: 0.3176
Epoch 2/2000
3/3 [==============================] - 0s 30ms/step - loss: 1.8947 - categorical_accuracy: 0.3059
Epoch 3/2000
3/3 [==============================] - 0s 31ms/step - loss: 1.1372 - categorical_accuracy: 0.4235
Epoch 4/2000
3/3 [==============================] - 0s 30ms/step - loss: 1.7855 - categorical_accuracy: 0.3647
Epoch 5/2000
3/3 [==============================] - 0s 40ms/step - loss: 1.7052 - categorical_accuracy: 0.4000
Epoch 6/2000
3/3 [==============================] - 0s 31ms/step - loss: 1.5881 - categorical_accuracy: 0.3647
Epoch 7/2000
3/3 [==============================] - 0s 35ms/step - loss: 1.1894 - categorical_accuracy: 0.4118
Epoch 8/2000
3/3 [==============================] - 0s 36ms/step - loss: 1.0740 - categorical_accuracy: 0.4353
Epoch 9/2000
3/3 [==============================] - 0s 35ms/step - loss: 1.1214 - categorical_accuracy: 0.3765

3/3 [==============================] - 0s 35ms/step - loss: 1.0429 - categorical_accuracy: 0.5294
Epoch 143/2000
3/3 [==============================] - 0s 29ms/step - loss: 1.0454 - categorical_accuracy: 0.4000
Epoch 144/2000
3/3 [==============================] - 0s 30ms/step - loss: 0.9542 - categorical_accuracy: 0.5765
Epoch 145/2000
3/3 [==============================] - 0s 32ms/step - loss: 0.9003 - categorical_accuracy: 0.5765
Epoch 146/2000
3/3 [==============================] - 0s 29ms/step - loss: 0.7838 - categorical_accuracy: 0.6706
Epoch 147/2000
3/3 [==============================] - 0s 31ms/step - loss: 5.3617 - categorical_accuracy: 0.4471
Epoch 148/2000
3/3 [==============================] - 0s 34ms/step - loss: 3.5493 - categorical_accuracy: 0.3294
Epoch 149/2000
3/3 [==============================] - 0s 33ms/step - loss: 1.3620 - categorical_accuracy: 0.3294
Epoch 150/2000
3/3 [==============================] - 0s 33ms/step - loss: 1.2683 - categorical_accuracy: 0.258

3/3 [==============================] - 0s 35ms/step - loss: 2.1477 - categorical_accuracy: 0.3176
Epoch 215/2000
3/3 [==============================] - 0s 31ms/step - loss: 1.7703 - categorical_accuracy: 0.3412
Epoch 216/2000
3/3 [==============================] - 0s 29ms/step - loss: 1.2077 - categorical_accuracy: 0.3882
Epoch 217/2000
3/3 [==============================] - 0s 29ms/step - loss: 1.2870 - categorical_accuracy: 0.3176
Epoch 218/2000
3/3 [==============================] - 0s 28ms/step - loss: 1.1630 - categorical_accuracy: 0.4118
Epoch 219/2000
3/3 [==============================] - 0s 33ms/step - loss: 1.2596 - categorical_accuracy: 0.3765
Epoch 220/2000
3/3 [==============================] - 0s 32ms/step - loss: 1.1618 - categorical_accuracy: 0.4118
Epoch 221/2000
3/3 [==============================] - 0s 33ms/step - loss: 1.1631 - categorical_accuracy: 0.3412
Epoch 222/2000
3/3 [==============================] - 0s 31ms/step - loss: 1.1809 - categorical_accuracy: 0.282

3/3 [==============================] - 0s 38ms/step - loss: 0.5646 - categorical_accuracy: 0.7529
Epoch 359/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.5680 - categorical_accuracy: 0.7412
Epoch 360/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.5700 - categorical_accuracy: 0.7294
Epoch 361/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.5762 - categorical_accuracy: 0.7412
Epoch 362/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.5718 - categorical_accuracy: 0.7294
Epoch 363/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.5880 - categorical_accuracy: 0.7176
Epoch 364/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.6758 - categorical_accuracy: 0.6941
Epoch 365/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.6144 - categorical_accuracy: 0.6941
Epoch 366/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.9651 - categorical_accuracy: 0.588

3/3 [==============================] - 0s 38ms/step - loss: 0.4184 - categorical_accuracy: 0.8588
Epoch 503/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.4162 - categorical_accuracy: 0.8471
Epoch 504/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.4341 - categorical_accuracy: 0.8118
Epoch 505/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.4177 - categorical_accuracy: 0.8235
Epoch 506/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.4399 - categorical_accuracy: 0.8353
Epoch 507/2000
3/3 [==============================] - 0s 33ms/step - loss: 0.4498 - categorical_accuracy: 0.8471
Epoch 508/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.4308 - categorical_accuracy: 0.8000
Epoch 509/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.4401 - categorical_accuracy: 0.8235
Epoch 510/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.4126 - categorical_accuracy: 0.835

...

3/3 [==============================] - 0s 35ms/step - loss: 0.0702 - categorical_accuracy: 0.9765
Epoch 1583/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.0579 - categorical_accuracy: 0.9765
Epoch 1584/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.0503 - categorical_accuracy: 1.0000
Epoch 1585/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.0512 - categorical_accuracy: 1.0000
Epoch 1586/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0548 - categorical_accuracy: 0.9882
Epoch 1587/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.0435 - categorical_accuracy: 1.0000
Epoch 1588/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0556 - categorical_accuracy: 0.9882
Epoch 1589/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0778 - categorical_accuracy: 0.9765
Epoch 1590/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0972 - categorical_accurac

3/3 [==============================] - 0s 35ms/step - loss: 0.0866 - categorical_accuracy: 0.9647
Epoch 1655/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0540 - categorical_accuracy: 0.9882
Epoch 1656/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.0539 - categorical_accuracy: 0.9882
Epoch 1657/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0359 - categorical_accuracy: 1.0000
Epoch 1658/2000
3/3 [==============================] - 0s 32ms/step - loss: 0.0416 - categorical_accuracy: 1.0000
Epoch 1659/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0391 - categorical_accuracy: 1.0000
Epoch 1660/2000
3/3 [==============================] - 0s 33ms/step - loss: 0.0324 - categorical_accuracy: 1.0000
Epoch 1661/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.0269 - categorical_accuracy: 1.0000
Epoch 1662/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.0236 - categorical_accurac

3/3 [==============================] - 0s 38ms/step - loss: 0.0046 - categorical_accuracy: 1.0000
Epoch 1727/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.0045 - categorical_accuracy: 1.0000
Epoch 1728/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.0044 - categorical_accuracy: 1.0000
Epoch 1729/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.0042 - categorical_accuracy: 1.0000
Epoch 1730/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.0043 - categorical_accuracy: 1.0000
Epoch 1731/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0043 - categorical_accuracy: 1.0000
Epoch 1732/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.0040 - categorical_accuracy: 1.0000
Epoch 1733/2000
3/3 [==============================] - 0s 47ms/step - loss: 0.0041 - categorical_accuracy: 1.0000
Epoch 1734/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.0041 - categorical_accurac

3/3 [==============================] - 0s 37ms/step - loss: 0.0020 - categorical_accuracy: 1.0000
Epoch 1799/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0020 - categorical_accuracy: 1.0000
Epoch 1800/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 1801/2000
3/3 [==============================] - 0s 47ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 1802/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 1803/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 1804/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 1805/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 1806/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0018 - categorical_accurac

3/3 [==============================] - 0s 35ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 1871/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 1872/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 1873/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 1874/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 1875/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 1876/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 1877/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 1878/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.0011 - categorical_accurac

In [ ]:
# x_train.shape

In [ ]:
# actions.shape[0]

In [22]:
res = model.predict(x_test)

1/1 [==============================] - 0s 380ms/step


In [23]:
actions[np.argmax(res[2])]

'thanks'

In [24]:
actions[np.argmax(y_test[2])]

'thanks'

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 59667

#### Model Saving...

In [27]:
model.save('action.keras')

In [17]:
model.load_weights('action.keras')

#### Model evaluation...

In [18]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [25]:
ypred = model.predict(x_train)

3/3 [==============================] - 1s 31ms/step


In [26]:
yreal = np.argmax(y_train, axis=1).tolist()
ypred = np.argmax(ypred, axis=1).tolist()

In [27]:
multilabel_confusion_matrix(yreal, ypred)

array([[[55,  0],
        [ 0, 30]],

       [[57,  0],
        [ 0, 28]],

       [[58,  0],
        [ 0, 27]]], dtype=int64)

In [28]:
accuracy_score(yreal, ypred)

1.0

#### Real-time predictions...

In [29]:
def prob_stats(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,50+num*40), (int(prob*100), 80+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 75+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [30]:
frames = []
sentence = []
predictions = []
threshold = 0.8
colors = [(245,117,16), (117,245,16), (16,117,245)]

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.6, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame, holistic) # Detecting
#           print(results)

        draw_landmarks(image, results)
    
        keypoints = extract_keypoints(results)
        frames.append(keypoints)
        frames = frames[-30:]
    
        if len(frames) == 30:
            res = model.predict(np.expand_dims(frames, axis=0))[0]
#             print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            image = prob_stats(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
        cv2.imshow('OpenCV Feed', image)
    
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 30ms/step


In [59]:
cap.release()

#### Miscelleaneous...


In [76]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Reshape

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(30, 1662, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Flatten())
model.add(Reshape((1024, -1)))

model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 28, 1660, 32)      320       
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 14, 830, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 12, 828, 64)       18496     
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 6, 414, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 4, 412, 64)        36928     
                                                                 
 flatten_7 (Flatten)         (None, 105472)          

In [79]:
model.fit(x_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
3/3 [==============================] - 4s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 2/1000
3/3 [==============================] - 3s 968ms/step - loss: 1.0985 - accuracy: 0.3412
Epoch 3/1000
3/3 [==============================] - 3s 977ms/step - loss: 1.0985 - accuracy: 0.3412
Epoch 4/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 5/1000
3/3 [==============================] - 3s 992ms/step - loss: 1.0985 - accuracy: 0.3412
Epoch 6/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 7/1000
3/3 [==============================] - 3s 984ms/step - loss: 1.0985 - accuracy: 0.3412
Epoch 8/1000
3/3 [==============================] - 3s 971ms/step - loss: 1.0985 - accuracy: 0.3412
Epoch 9/1000
3/3 [==============================] - 3s 978ms/step - loss: 1.0985 - accuracy: 0.3412
Epoch 10/1000
3/3 [==============================] - 3s 975ms/step - loss: 1.0985 - accuracy: 0.3412
Epoch 11

3/3 [==============================] - 4s 1s/step - loss: 1.0986 - accuracy: 0.3412
Epoch 168/1000
3/3 [==============================] - 5s 2s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 169/1000
3/3 [==============================] - 4s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 170/1000
3/3 [==============================] - 4s 2s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 171/1000
3/3 [==============================] - 4s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 172/1000
3/3 [==============================] - 4s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 173/1000
3/3 [==============================] - 4s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 174/1000
3/3 [==============================] - 4s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 175/1000
3/3 [==============================] - 4s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 176/1000
3/3 [==============================] - 4s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 177/1000
3/3 [=====

3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 332/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 333/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 334/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 335/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 336/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 337/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 338/1000
3/3 [==============================] - 4s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 339/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 340/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 341/1000
3/3 [=====

3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 496/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 497/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 498/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 499/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 500/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 501/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 502/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 503/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 504/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 505/1000
3/3 [=====

3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 660/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 661/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 662/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 663/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 664/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 665/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 666/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 667/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 668/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 669/1000
3/3 [=====

3/3 [==============================] - 4s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 824/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 825/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 826/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 827/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 828/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 829/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 830/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 831/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 832/1000
3/3 [==============================] - 3s 1s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 833/1000
3/3 [=====

3/3 [==============================] - 8s 3s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 988/1000
3/3 [==============================] - 8s 3s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 989/1000
3/3 [==============================] - 8s 3s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 990/1000
3/3 [==============================] - 8s 3s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 991/1000
3/3 [==============================] - 8s 3s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 992/1000
3/3 [==============================] - 8s 3s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 993/1000
3/3 [==============================] - 8s 3s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 994/1000
3/3 [==============================] - 8s 3s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 995/1000
3/3 [==============================] - 8s 3s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 996/1000
3/3 [==============================] - 8s 3s/step - loss: 1.0985 - accuracy: 0.3412
Epoch 997/1000
3/3 [=====

In [80]:
res1 = model.predict(x_test)

1/1 [==============================] - 2s 2s/step


In [81]:
actions[np.argmax(res1[2])]

'iloveyou'

In [82]:
actions[np.argmax(y_test[2])]

'thanks'

In [83]:
model.save('proto_hybridCNN-RNN.keras')